## Line-plane Intersction

In [34]:
import numpy as np
import math

In [35]:
def LinePlaneCollision(planeNormal, planePoint, rayDirection, rayPoint, epsilon=1e-6):
    ndotu = planeNormal.dot(rayDirection)
    if abs(ndotu) < epsilon:
        raise RuntimeError("no intersection or line is within plane")
 
    w = rayPoint - planePoint
    si = -planeNormal.dot(w) / ndotu
    Psi = w + si * rayDirection + planePoint
    return Psi

In [36]:
#Define plane
planeNormal = np.array([0, 0, 1])
planePoint = np.array([0, 0, 5]) #Any point on the plane

#Define ray
rayDirection = np.array([0, -1, -1])
rayPoint = np.array([0, 0, 10]) #Any point along the ray

Psi = LinePlaneCollision(planeNormal, planePoint, rayDirection, rayPoint)
print ("intersection at", Psi)

intersection at [ 0. -5.  5.]


## Ray from Camera

In [37]:
# camera setup
camW = 1920  # units: pixel (mm when calculated)
camH = 1080  # units: pixel (mm when calculated)
camAngleW = 110  # units: degree

camD = (camW/2)/math.tan(math.radians(camAngleW/2))  # units: mm
print ('camD = ', camD)

camD =  672.1992366813215


In [38]:
# https://stackoverflow.com/questions/6802577/rotation-of-3d-vector

def RotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def RotPtAxis(pt, axis, theta):
    # theta in radians
    return np.dot(RotationMatrix(axis, theta), pt)

pt = [3, 5, 0]
axis = [4, 4, 1]
theta = 1.2 

print(RotPtAxis(pt, axis, theta)) 
# [ 2.74911638  4.77180932  1.91629719]

[2.74911638 4.77180932 1.91629719]


In [46]:
print(math.degrees(1.2))

68.75493541569878


In [39]:
camPos = [0, 0, 3000]  # units: mm
camRot = [-25, 5, 45]  # local euler angle(x, y, z); units: degree

In [53]:
pixPos = [400, 600] # units: pixel (mm when calculated)

def CamPixToWorldPos(pixPos, camPos, camRot, camW, camH, camAngleW):
    '''
    camRot in degrees
    pixPos in pixels (mm when calculated), [0,0]is top right of the image
    camPos in mm
    camW, camH in pixels (mm when calculated)
    camAngleW in degrees
    '''
    camD = (camW/2)/math.tan(math.radians(camAngleW/2))
    camPos = np.asarray(camPos)
    camRot = np.asarray(camRot)
    # start with cam at [0,0,0], fwd facing, no rot, get pix world pos
    pt = np.array([pixPos[0]-camW/2, camD, camH/2-pixPos[1]])
    # rotate pt with cam at [0,0,0]
    ## local z axis
    localZAxis = [0,0,1]
    pt = rotPtAxis(pt, localZAxis, math.radians(camRot[2]))
    ## local x axis
    localXAxis = [1,0,0]
    localXAxis = rotPtAxis(localXAxis, localZAxis, math.radians(camRot[2]))
    pt = rotPtAxis(pt, localXAxis, math.radians(camRot[0]))
    ## local y axis
    localYAxis = [0,1,0]
    localYAxis = rotPtAxis(localYAxis, localZAxis, math.radians(camRot[2]))
    localYAxis = rotPtAxis(localYAxis, localXAxis, math.radians(camRot[0]))
    pt = rotPtAxis(pt, localYAxis, math.radians(camRot[1]))
    # move pt to cam pos
    pt = pt + camPos
    return pt

print(CamPixToWorldPos(pixPos, camPos, camRot, camW, camH, camAngleW))

[-825.67727087   29.33591129 2705.97914618]


# Camera Pixel and Ground Plane Intersection

In [51]:
def CamPixToPlaneIntersection(pixPos, camPos, camRot, camW, camH, camAngleW, planeNormal, planePoint):
    camPos = np.asarray(camPos)
    camRot = np.asarray(camRot)
    pixWorldPos = CamPixToWorldPos(pixPos, camPos, camRot, camW, camH, camAngleW)
    rayDirection = pixWorldPos - camPos
    rayPoint = camPos
    pt = LinePlaneCollision(planeNormal, planePoint, rayDirection, rayPoint)
    return pt

In [55]:
#test 01

# camera setup
camW = 1920  # units: pixel (mm when calculated)
camH = 1080  # units: pixel (mm when calculated)
camAngleW = 110  # units: degree
# camera pos rot
camPos = [0, 0, 3000]  # units: mm
camRot = [-25, 5, 45]  # local euler angle(x, y, z); units: degree
# pixel info
pixPos = [400, 600] # units: pixel (mm when calculated)
# plane info
planeNormal = np.array([0, 0, 1])
planePoint = np.array([0, 0, 0]) #Any point on the plane

# test
pt = CamPixToPlaneIntersection(pixPos, camPos, camRot, camW, camH, camAngleW, planeNormal, planePoint)
print('intersection point: ', pt)

intersection point:  [-8424.68070013   299.32480204     0.        ]


In [56]:
#test 02

# camera setup
camW = 800  # units: pixel (mm when calculated)
camH = 600  # units: pixel (mm when calculated)
camAngleW = 95  # units: degree
# camera pos rot
camPos = [5503, 2405, 2837]  # units: mm
camRot = [-21.9, -12.5, -45]  # local euler angle(x, y, z); units: degree
# pixel info
pixPos = [135, 402] # units: pixel (mm when calculated)
# plane info
planeNormal = np.array([0, 0, 1])
planePoint = np.array([0, 0, 0]) #Any point on the plane

# test
pt = CamPixToPlaneIntersection(pixPos, camPos, camRot, camW, camH, camAngleW, planeNormal, planePoint)
print('intersection point: ', pt)

intersection point:  [5822.06714297 6086.97727726    0.        ]


### tested with rhino 3D model, results are precisely correct. 